In [1]:
!pip install backtesting ta-lib seaborn matplotlib

In [3]:
import numpy as np
import pandas as pd
import talib
from backtesting import Backtest, Strategy

In [15]:
import mysql.connector
from mysql.connector import Error
conn = None
try:
    # Conéctate a la base de datos MySQL
    conn = mysql.connector.connect(
        host='localhost',
        database='backtesting',
        user='pruebas',
        password='contraseñaprueba'
    )

    if conn.is_connected():
        # Ejecuta la consulta para obtener los datos
        query = "SELECT * FROM btc_usdt_1h LIMIT 350"
        df_btc = pd.read_sql(query, conn)

        print("Datos obtenidos de la base de datos:")
        print(df_btc.head())

except Error as e:
    print(f"Error: {e}")
finally:
    if conn is not None and conn.is_connected():
        conn.close()
        print("Conexión a MySQL cerrada.")

#para cargar correctamente los datos a la estrategia las columnas deben estar con mayusculas
df_btc.columns = [col.capitalize() for col in df_btc.columns]
df_btc

Datos obtenidos de la base de datos:
            Timestamp      Open      High       Low     Close      Volume
0 2024-05-04 00:00:00  63613.92  63932.61  63606.62  63892.04   451.84843
1 2024-05-04 09:00:00  63160.95  63300.00  63055.55  63179.80   576.45042
2 2024-05-04 10:00:00  63179.80  64497.99  63110.66  64420.91  2955.45058
3 2024-05-04 11:00:00  64420.90  64540.00  63658.79  63873.31  2394.63450
4 2024-05-04 12:00:00  63873.30  64098.63  63758.51  64066.35  1062.45014
Conexión a MySQL cerrada.


/tmp/ipykernel_464545/2280991615.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_btc = pd.read_sql(query, conn)


,Timestamp,Open,High,Low,Close,Volume
0,2024-05-04 00:00:00,63613.92,63932.61,63606.62,63892.04,451.84843
1,2024-05-04 09:00:00,63160.95,63300.00,63055.55,63179.80,576.45042
2,2024-05-04 10:00:00,63179.80,64497.99,63110.66,64420.91,2955.45058
3,2024-05-04 11:00:00,64420.90,64540.00,63658.79,63873.31,2394.63450
4,2024-05-04 12:00:00,63873.30,64098.63,63758.51,64066.35,1062.45014
...,...,...,...,...,...,...
345,2024-05-18 17:00:00,66888.17,66970.00,66750.00,66866.51,501.97204
346,2024-05-18 18:00:00,66866.52,66910.00,66779.95,66796.36,230.01863
347,2024-05-18 19:00:00,66796.35,66942.94,66771.76,66912.62,224.34717
348,2024-05-18 20:00:00,66912.62,67014.88,66880.00,66964.01,285.93209


In [5]:
class SMAPullbackStrategy(Strategy):
    n = 20

    def init(self):
        close = self.data.Close
        self.sma = self.I(talib.SMA, close, self.n)
    
    def next(self):
        price = self.data.Close[-1]
        sma = self.sma[-1]
        yesterdays_low = self.data.Low[-1]
        stop_loss = yesterdays_low
        take_profit = yesterdays_low + 1.5 *(price - stop_loss)

        if self.position.is_long:
            if price <= stop_loss or price >= take_profit:
                self.position.close()

        elif price > sma:
            print(stop_loss, ", ", take_profit)
            try:
                self.buy(sl=stop_loss, tp=take_profit)
            except:
                pass
        

        

In [6]:
bt = Backtest(df_btc, SMAPullbackStrategy, cash=1000000, commission=0.002)
stats = bt.run()
#bt.plot()

53357.0 ,  53999.33
53617.61 ,  54118.34
53661.67 ,  55593.745
54080.0 ,  54948.5
54415.06 ,  58999.494999999995
53879.0 ,  57055.895000000004
54167.19 ,  58966.905
56850.0 ,  64083.63
61322.22 ,  65759.775
63481.4 ,  67261.41500000001
62000.0 ,  62289.725000000006
60000.0 ,  61032.33
59562.15 ,  62149.05
59510.63 ,  61523.015
60650.0 ,  64293.17
59817.55 ,  60584.439999999995
57820.0 ,  61953.835
60174.81 ,  63293.16
59945.36 ,  61977.020000000004
60624.68 ,  64517.645
60382.76 ,  64153.340000000004
61322.78 ,  64248.994999999995
63273.58 ,  69651.955
62822.9 ,  65912.195
62278.0 ,  65044.345
63576.27 ,  66490.515
63400.0 ,  63710.11
47920.42 ,  52298.005
37026.73 ,  43848.009999999995
41645.85 ,  44937.06
42666.0 ,  44731.485
43117.92 ,  45000.12
43174.01 ,  43656.155
41938.51 ,  42591.340000000004
41688.88 ,  42482.365000000005
41870.0 ,  42145.490000000005
41550.56 ,  43028.630000000005
42427.03 ,  45603.775
37450.17 ,  46014.915
38656.45 ,  43584.34
38828.48 ,  42256.759999999995


/tmp/ipykernel_464545/2355427670.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_btc, SMAPullbackStrategy, cash=1000000, commission=0.002)


In [7]:
print(stats)

Start                                     0.0
End                                     349.0
Duration                                349.0
Exposure Time [%]                        36.0
Equity Final [$]                 487978.69426
Equity Peak [$]                  1061347.1083
Return [%]                         -51.202131
Buy & Hold Return [%]              -57.438756
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -55.032351
Avg. Drawdown [%]                  -30.169898
Max. Drawdown Duration                  316.0
Avg. Drawdown Duration                  162.0
# Trades                                 83.0
Win Rate [%]                         50.60241
Best Trade [%]                       5.265094
Worst Trade [%]                    -13.402643
Avg. Trade [%]                    

In [8]:
class SMAPullbackStrategy(Strategy):
    stop_loss = 0
    n1 = 20

    def init(self):
        price = self.data.Close
        self.sma = self.I(talib.SMA, price, self.n1)
    
    def next(self):
        price = self.data.Close[-1]
        low_price_prev_day = self.data.Low[-1]

        if price > self.sma[-1]:
            self.stop_loss = low_price_prev_day

            if price <= 2.4 * self.stop_loss:
                self.buy()
            

In [9]:
bt = Backtest(df_btc, SMAPullbackStrategy, cash=1000000, commission=0.002)

output = bt.optimize(maximize='Equity Final [$]', n1=range(5,40,1))

stats = bt.run()
bt.plot()

/tmp/ipykernel_464545/3251659232.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_btc, SMAPullbackStrategy, cash=1000000, commission=0.002)


GridPlot(id='p1466', ...)

Abriendo en una sesión existente del navegador


In [22]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG


class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 9)
        self.ma2 = self.I(SMA, price, 38)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(df_btc, SmaCross,cash=1000000, commission=.002,
              exclusive_orders=True)
stats = bt.run()
print(stats)

Start                                     0.0
End                                     349.0
Duration                                349.0
Exposure Time [%]                   83.428571
Equity Final [$]                 962356.22522
Equity Peak [$]                  1010760.9646
Return [%]                          -3.764377
Buy & Hold Return [%]                4.706768
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -8.088795
Avg. Drawdown [%]                   -3.809558
Max. Drawdown Duration                  227.0
Avg. Drawdown Duration              96.666667
# Trades                                 10.0
Win Rate [%]                             40.0
Best Trade [%]                       4.170268
Worst Trade [%]                     -3.770902
Avg. Trade [%]                    

/tmp/ipykernel_464545/2091554013.py:20: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_btc, SmaCross,cash=1000000, commission=.002,


In [17]:
bt.plot()

GridPlot(id='p3365', ...)

Abriendo en una sesión existente del navegador


In [19]:
df_btc.tail(50)

,Timestamp,Open,High,Low,Close,Volume
300,2024-05-16 20:00:00,65186.84,65290.00,65050.00,65273.90,581.14820
301,2024-05-16 21:00:00,65273.90,65500.00,65242.09,65461.99,579.45707
302,2024-05-16 22:00:00,65461.99,65480.35,65140.61,65418.97,580.56491
303,2024-05-16 23:00:00,65418.98,65422.03,65187.50,65235.21,688.52174
304,2024-05-17 00:00:00,65235.21,65508.70,65235.20,65508.69,591.23809
305,2024-05-17 01:00:00,65508.70,65510.22,65106.38,65340.84,520.23452
306,2024-05-17 02:00:00,65340.83,65450.07,65188.00,65386.01,370.41889
307,2024-05-17 03:00:00,65386.00,65901.46,65354.23,65584.53,1533.13211
308,2024-05-17 04:00:00,65584.53,65630.70,65367.51,65451.81,567.84995
309,2024-05-17 05:00:00,65451.81,65700.00,65436.00,65700.00,472.55691
